# "End to End Deep Learning Project with fastai and FastAPI"
> "A quick and simple tutorial on how to go about creating your own state-of-the-art image classifier and deploy it using fastai, FastAPI, Docker and Heroku"

- toc: true
- author: Sayantan Karmakar
- badges: true
- hide_binder_badge: true
- comments: true
- categories: [fastai, FastAPI, Docker, Heroku, MNIST, CNN]
- show_tags: true

# Acknowledgment

Before starting on with this tutorial, I would like to mention a few names and resources without which this wouldn't have been possible.

Firstly, I would like to thank [Jeremy Howard](https://twitter.com/jeremyphoward), [Sylvain Gugger](https://twitter.com/GuggerSylvain) and [Rachel Thomas](https://twitter.com/math_rachel) for creating fastai, along with its [course](https://course.fast.ai) and their new book, [Deep Learning for Coders with fastai and PyTorch](https://www.oreilly.com/library/view/deep-learning-for/9781492045519/) which makes Deep Learning easier and accessible than ever. Then I would like to thank [Hamel Husain](https://twitter.com/HamelHusain) for creating fastpages, with which this entire blog post was made. It's such a great tool, and I would encourage everyone thinking of starting a blog to check it out!

And finally, I would like to thank the entire fastai community, for helping me out with my silly doubts on the forums and discord channel, special mention to [Zach Mueller](https://twitter.com/TheZachMueller)!

# Introduction

In this tutorial, we'll look at how easy and fast training a **state-of-the-art** Convolutional Neural Network on the famous **MNIST** dataset can be using fastai. Then we'll use this model to create an API which when provided with an image input, will send back a response to the client. We will then deploy this API using Docker and Heroku to create your very own fully functioning web app.

Since this tutorial is going to be more focused on how to train a Deep Learning model, and deploy it for a web app, I expect the readers to have a basic understanding of how Neural Networks work, along with its underlying concepts such as **Backpropagation**, **Gradient Descent** etc.

If you're unfamiliar with these topics, I highly recommend you to take the [fastai course](https://course.fast.ai).

# Getting Started

## Python packages

To get started, we need to install [PyTorch](https://pytorch.org/get-started/locally/), [fastai](https://docs.fast.ai) and [FastAPI](https://fastapi.tiangolo.com/) on our system. Ideally, I would recommend everyone to use Anaconda/Miniconda to manage your Python packages and environments.

Assuming you have Anaconda/Miniconda installed, the following lines of code would install the required packages for this tutorial - 

```
conda install pytorch torchvision cudatoolkit=10.2 -c pytorch

pip install --upgrade fastai

pip install fastapi[all]
```

## Docker and Heroku

Installing is a pretty straightforward process that you can follow in their [documentation](https://docs.docker.com/get-docker/), and if you're using Windows or Mac, I highly recommend installing the **Docker Desktop**. Once you're done installing it, you'll also have to sign up, which they ask you during the installation process.

You'll also need to install the **Heroku CLI**, which again, you can follow their [documentation](https://devcenter.heroku.com/articles/heroku-cli). Make sure you sign up on Heroku and have an account ready, as you'll be needing it later in the tutorial.

## Development environment

If you've followed the fastai course, you would know that the best way to write fastai code is in a IPython notebook environment, as fastai provides many additional features which help your training process much easier and insightful. Hence I would recommend everyone to either install [Jupyterlab](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html) on their systems, or check out [Google Colab](https://colab.research.google.com/), which provides you with a Notebook environment in your browser, along with **free GPU support!!**.

For the rest of the tutorial, excluding the fastai code, I would recommend any text editor of your choice, but **VSCode** would be preferred as it provides with many features like type-checking for FastAPI.

> Tip: If your system doesn't have GPU support, I highly recommend using **Google Colab** as it will make training your model much faster!

# Preparing our Model

## Importing libraries

Believe it or not, but we only need a **single** import statement thing to train our entire model.

In [1]:
from fastai.vision.all import *

Now some of you might say it is a _bad practice_ to `import *` in a Python project, but that is the beauty of fastai, which keeps in mind all the issues that would come in doing so, and is written in such a way that it is ***recommended*** to import it like this!

## Getting our data

To make things easy, I chose on purpose to use the **[MNIST](http://yann.lecun.com/exdb/mnist/)** dataset, which is provided by fastai out-of-the-box. We just have to download it, and untar the downloaded file. This can be easily done with the following code.

In [ ]:
path = untar_data(URLs.MNIST)

In [10]:
#hide
Path.BASE_PATH = path

> Note: Depending on your system, this `path` object will be of either `class 'pathlib.PosixPath'` or `class 'pathlib.WindowsPath'`. This is a built-in feature of Python, which I would say is one of the most useful features I've come across while learning fastai and would highly recommend you to check out the **`pathlib`** library. 

Before we start creating our datasets and dataloaders, we should always check how our data is organized to get an idea of how we're supposed to label and divide our data.

In [11]:
path.ls()

(#2) [Path('testing'),Path('training')]

In [13]:
(path/'training').ls()

(#10) [Path('training/0'),Path('training/1'),Path('training/2'),Path('training/3'),Path('training/4'),Path('training/5'),Path('training/6'),Path('training/7'),Path('training/8'),Path('training/9')]

In [14]:
(path/'testing').ls()

(#10) [Path('testing/0'),Path('testing/1'),Path('testing/2'),Path('testing/3'),Path('testing/4'),Path('testing/5'),Path('testing/6'),Path('testing/7'),Path('testing/8'),Path('testing/9')]

In [15]:
(path/'training/0').ls()

(#5923) [Path('training/0/1.png'),Path('training/0/1000.png'),Path('training/0/10005.png'),Path('training/0/10010.png'),Path('training/0/10022.png'),Path('training/0/10025.png'),Path('training/0/10026.png'),Path('training/0/10045.png'),Path('training/0/10069.png'),Path('training/0/10071.png')...]

As we can see from the above blocks of code, our data is first divided into `training` and `testing` directories, which then includes a directory for each digit, containing images of that particular digit.

Hence, we see that the labels of the images are their **parent directory** names. Later in this section, we'll see how easy it is to label your images using fastai!

## Creating our DataBlock

Now that we had a look at our data, and know how to go about labelling them. We now have to create, what we call `Dataset` and `Dataloader` in PyTorch, which are the two main classes for representing and accessing training or validation data.

* `Dataset`: A collection that returns a tuple of your independent and dependent variable for a single item.
* `DataLoader`: An iterator that provides a stream of mini-batches, where each mini-batch is a tuple of a batch of independent variables and a batch of dependent variables.

Since a `DataLoader` builds on top of a `Dataset` and adds additional functionality to it (collating multiple items into a mini-batch), it’s often easiest to start by creating and testing `Datasets`, and then look at `DataLoaders` after that’s working.

For example,

In [24]:
x = list(np.arange(0,26))
y = [chr(ord('a') + i)  for i in x]
dset = list(zip(x,y))

for i in range(3):
    print(dset[i])

(0, 'a')
(1, 'b')
(2, 'c')


Hence, the `dset` object in the code above, is a valid `Dataset` in PyTorch as we can index it, and it returns a tuple of the independent and dependent variable, which in this case is integers from **0 to 26** and letters **a to z** respectively.

Now if you wish to create a `DataLoader` for this `Dataset`, you can use the following function provided by PyTorch:

In [26]:
dloader = torch.utils.data.DataLoader(dset, batch_size=4,
                                      shuffle=True, drop_last=True)
for i in dloader:
    print(i)

[tensor([21, 25, 24, 23], dtype=torch.int32), ('v', 'z', 'y', 'x')]
[tensor([ 6, 12, 20,  5], dtype=torch.int32), ('g', 'm', 'u', 'f')]
[tensor([ 4, 22,  8, 15], dtype=torch.int32), ('e', 'w', 'i', 'p')]
[tensor([18, 11, 13,  7], dtype=torch.int32), ('s', 'l', 'n', 'h')]
[tensor([10, 19,  3,  1], dtype=torch.int32), ('k', 't', 'd', 'b')]
[tensor([ 2,  0, 16, 17], dtype=torch.int32), ('c', 'a', 'q', 'r')]


The parameters of the above function are pretty self-explanatory except maybe `drop_last`, which allows you to drop the last batch of data if it is not of the given `batch_size` as in some cases it is important to have all your batches of similar sizes.